In [3]:
import pandas as pd

income = pd.read_csv("adult.data",header=None)
income.columns = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "high_income"]
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
categories = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for i in categories:
    col = pd.Categorical.from_array(income[i])
    income[i] = col.codes
    
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


### Split into private incomes and public incomes

In [5]:
private_incomes = income[income["workclass"] == 4]
public_incomes = income[income["workclass"] != 4]

### Calculate entropy of the "high income" column

In [6]:
import math

count = income.shape[0]
low_income_count = income.high_income.value_counts()[0]
high_income_count = income.high_income.value_counts()[1]

income_entropy = -(low_income_count/count * math.log(low_income_count/count, 2) + high_income_count/count * math.log(high_income_count/count, 2))
print(income_entropy)

0.796383955202


### Calculate information gain for splitting on "age" column

In [7]:
import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas Series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column.
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability.
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0.
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy.
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

income_entropy = calc_entropy(income.high_income)
print("Entropy for high_income:", income_entropy)

median = income.age.median()
print("Median age:", median)

left_split = income[income.age <= median]
right_split = income[income.age > median]

lp = left_split.shape[0] / income.shape[0]
rp = right_split.shape[0] / income.shape[0]

age_information_gain = income_entropy - ((lp * calc_entropy(left_split.high_income)) + (rp * calc_entropy(right_split.high_income)))
print("Information gain for age column:", age_information_gain)

Entropy for high_income: 0.796383955202
Median age: 37.0
Information gain for age column: 0.0470286613047


### Calculate information gain for all columns and find the one with the highest information gain

In [8]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a dataset, column to split on, and target.
    """
    # Calculate original entropy.
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting.
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data based on the median.
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits, and calculate the subset entropy.
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain.
    return original_entropy - to_subtract

# Verify that our answer is the same as in the last screen.
print("Information gain for age column:", calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

information_gains = {}
for c in columns:
    information_gains[c] = calc_information_gain(income, c, "high_income")
    
print("\nInformation gain for each column:", information_gains)
highest_gain = max(information_gains, key=lambda p: information_gains[p])
print("\nColumn containing the highest information gain:", highest_gain)

Information gain for age column: 0.0470286613047

Information gain for each column: {'occupation': 0.0015822303843424645, 'hours_per_week': 0.040622468671234868, 'native_country': 0.00013457344495848567, 'marital_status': 0.1114272573715438, 'relationship': 0.047362416650269412, 'workclass': 0.0068109840543966182, 'sex': 0.0, 'education_num': 0.065012984132774232, 'race': 0.0, 'age': 0.047028661304691965}

Column containing the highest information gain: marital_status


### Create a function for determining best column to split on

In [9]:
def find_best_column(data, target_name, columns):
    information_gains = {}
    for c in columns:
        information_gains[c] = calc_information_gain(data, c, target_name)
    return max(information_gains, key=lambda p: information_gains[p])

# A list of columns to potentially split income with.
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

income_split = find_best_column(income, "high_income", columns)
print("Best column for split:", income_split)

Best column for split: marital_status


### ID3 algorithm implementation

In [10]:
# We'll use lists to store our labels for nodes (when we find them).
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this.
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a Series.
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        # Insert code here to append 1 to label_1s or 0 to label_0s based on what we should label the node.
        # See lines 2 and 3 in the algorithm.
        if unique_targets[0] == 1:
            label_1s.append(1)
        else:
            label_0s.append(0)
        
        # Returning here is critical -- if we don't, the recursive tree will never finish, and run forever.
        # See our example above for when we returned.
        return
    
    # Find the best column to split on in our data.
    best_column = find_best_column(data, target, columns)
    # Find the median of the column.
    column_median = data[best_column].median()
    
    # Create the two splits.
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively.
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch.
        id3(split, target, columns)
    
# Create the dataset that we used in the example in the last screen.
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data.
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly.
id3(data, "high_income", ["age", "marital_status"])

#### ID3 recursive algorithm implementation

In [11]:
# Create a dictionary to hold the tree.  This has to be outside the function so we can access it later.
tree = {}

# This list will let us number the nodes.  It has to be a list so we can access it inside the function.
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    # Assign the number key to the node dictionary.
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        tree["label"] = unique_targets[0]
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here to assign the "column" and "median" fields to the node dictionary.
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly.
id3(data, "high_income", ["age", "marital_status"], tree)

###

In [12]:
def print_with_depth(string, depth):
    # Add space before a string.
    prefix = "    " * depth
    # Print a string, appropriately indented.
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of label in the tree.
    if "label" in tree:
        # If there's a label, then this is a leaf, so print it and return.
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion.
        return
    # Print information about what the node is splitting on.
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches.
    branches = [tree["left"], tree["right"]]
        
    for branch in branches:
        print_node(branch, depth + 1)

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


#### Automatic predictions

In [13]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    # Insert code here to check if row[column] is less than or equal to median
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    # If it's greater, return the result of predicting on the right branch of the tree
    # Remember to use the return statement to return the result!
    if row[column] <= median:
        return predict(tree["left"], row)
    else:
        return predict(tree["right"], row)

# Print the prediction for the first row in our data.
print(predict(tree, data.iloc[0]))

0


#### Multiple predictions

In [16]:
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data.
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)
print(predictions)

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64


# Using scikit-learn

In [18]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with.
# All columns have been converted to numeric.
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier.
# Set random_state to 1 to keep results consistent.
clf = DecisionTreeClassifier(random_state=1)

# The variable income is loaded, and contains all the income data.
clf.fit(income[columns], income.high_income)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

#### Split the data into train and test sets

In [20]:
import numpy
import math

# Set a random seed so the shuffle is the same every time.
numpy.random.seed(1)

# Shuffle the rows.  This first permutes the index randomly using numpy.random.permutation.
# Then, it reindexes the dataframe with this.
# The net effect is to put the rows into random order.
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row,:]
test = income.iloc[train_max_row:,:]

#### Compute the error for the test set

In [22]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])

error = roc_auc_score(test.high_income, predictions) 
print(error)

0.69237495612


#### Compute error for training set to make sure it isn't overfitting

In [23]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train.high_income, predictions) )

0.945777105887


#### Build a shallower tree

In [28]:
# Decision trees model from the last screen.

# min_samples_split -- The minimum number of rows needed in a node before it can be split. 
# For example, if this is set to 2, then nodes with 2 rows won't be split, and will become leaves instead.

# max_depth -- this globally restricts how deep the tree can go.
clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)

clf.fit(train[columns], train["high_income"])

train_predictions = clf.predict(train[columns])
test_predictions = clf.predict(test[columns])
train_auc = roc_auc_score(train.high_income, train_predictions)
test_auc = roc_auc_score(test.high_income, test_predictions) 
print("Training AUC error:", train_auc)
print("Testing AUC eorr:", test_auc)

Training AUC error: 0.749285076088
Testing AUC eorr: 0.75043213051


#### High bias can cause underfitting -- 
if a model is consistently failing to predict the correct value, it may be that it is too simple to actually model the data.

#### High variance can cause overfitting -- 
if a model is very susceptible to small changes in the input data, and changes its predictions massively, then it is likely fitting itself to quirks in the training data, and not making a generalizable model.

#### Add a column of random values to induce variance

In [30]:
numpy.random.seed(1)

# Generate a column with random numbers from 0 to 4.
income["noise"] = numpy.random.randint(4, size=income.shape[0])

# Adjust columns to include the noise column.
columns = ["noise", "age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Make new train and test sets.
train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# Initialize the classifier.
clf = DecisionTreeClassifier(random_state=1)

clf.fit(train[columns], train["high_income"])

train_predictions = clf.predict(train[columns])
test_predictions = clf.predict(test[columns])
train_auc = roc_auc_score(train.high_income, train_predictions)
test_auc = roc_auc_score(test.high_income, test_predictions) 
print("Training AUC error:", train_auc)
print("Testing AUC eorr:", test_auc)

Training AUC error: 0.975645926216
Testing AUC eorr: 0.698333590245


#### Advantages to decision trees:
* Easy to interpret
* Relatively fast to fit and make predictions
* Able to handle multiple types of data
* Can pick up nonlinearities in data, and are usually fairly accurate

#### Disadvantages:
* Overfitting
* ![Decision Tree vs. Linear Regression](decisionTree_linearRegression.png)